Secondary structure metrics with BioPython/DSSP 

In [10]:
import os

from Bio import BiopythonWarning
import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore', BiopythonWarning)

from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
from Bio.PDB.DSSP import ss_to_index

import functools
from collections import Counter
import pandas as pd
import numpy as np

from pathlib import Path
#from multiprocessing import Pool

In [11]:
#cpus = 80

In [22]:
dataset_name = 'vhh_twist'
# structure_path = '/data/localhost/gordon/TNP_Project/results/debugged_nbb2_results/new_'+dataset_name+'_output/Raw_Model_Outputs/'
# structures = [str(s).split('/')[-1] for s in list(Path(structure_path).glob('**/*.pdb'))]

structure_path = '/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/reduced_strucs/reduced_'+dataset_name+'/'
structures = os.listdir(structure_path)
#files = [structure_path + structure for structure in structures]


In [23]:
def dssp_to_dict(structure, path = structure_path):

    try:
        #print(structure[:-4])
        p = PDBParser(QUIET=True)
        path_to_pdb = path + structure
        #print(path_to_pdb)
        model = p.get_structure("", path_to_pdb)[0] # GG modded here bc nested dirs
        #print(model)
        dssp = DSSP(model, path_to_pdb)
        #print(dssp)

        data = np.array(list(dict(dssp).values()))

        init_c = Counter({
            'H': 0,
            'B': 0,
            'E': 0,
            'G': 0,
            'I': 0,
            'T': 0,
            'S': 0,
            '-': 0
        })

        get_c = Counter(data[..., 2])

        ss_c = dict(functools.reduce(lambda a, b: a.update(b) or a, [init_c, get_c], Counter()))

        values = np.mean(data[..., 3:].astype(np.float32), axis=0)

        ss_d = {}
        keys = ['relative_ASA', 'phi', 'psi', 'NH_O_1_relidx', 
                'NH_O_1_energy', 'O_NH_1_relidx', 'O_NH_1_energy',
                'NH_O_2_relidx', 'NH_O_2_energy', 'O_NH_2_relidx', 
                'O_NH_2_energy']

        ss_d.update(ss_c)

        for n, key in enumerate(keys):
            ss_d.update({key: values[n]})
        
        return ss_d

    except Exception:
        print('Error:', structure[:-4])
        ss_d = {
            'H': 0,
            'B': 0,
            'E': 0,
            'G': 0,
            'I': 0,
            'T': 0,
            'S': 0,
            '-': 0,
            'relative_ASA': 0, 
            'phi': 0, 
            'psi': 0, 
            'NH_O_1_relidx': 0, 
            'NH_O_1_energy': 0,
            'O_NH_1_relidx': 0, 
            'O_NH_1_energy': 0,
            'NH_O_2_relidx': 0,
            'NH_O_2_energy': 0, 
            'O_NH_2_relidx': 0, 
            'O_NH_2_energy': 0
        }
        return ss_d

In [24]:
all_results = dict()
for structure in structures:
    results = dssp_to_dict(structure, path = structure_path)
    all_results[structure[:-4]] = results

In [25]:
len(all_results)

108

In [26]:
df = pd.DataFrame(all_results).reset_index()
#print(df.shape)

#df.insert(0, 'ID', [s[:-4] for s in structures])
# df.to_csv('./ABB_paired_dssp_2.csv', index=None)
#print(df.shape)

In [27]:
df

,index,seq_TBC-101-VA10040_VHH,seq_D8D85764-3574-KV-cFR2muts,seq_Ozoralizumab_VHH1,seq_D9D88792-5767-N,seq_Brivekimig2_VHH1,seq_TBC-107-11_VHH,seq_TBC-106-01_VHH,seq_D9D88792-8800-KV,seq_Rimteravimab_VHH1,...,seq_TBC-105-01_VHH,seq_D12D77826-11122-KV,seq_Enristomig_VHH2,seq_TBC-101-VA10006_VHH,seq_Envafolimab_VHH1,seq_TBC-103-VA10005_VHH,seq_D9D88792-13450-N-cFR2muts,seq_TBC-107-01_VHH,seq_D9D88792-13450-KV,seq_TBC-109-VA10008_VHH
0,H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
2,E,57.000000,56.000000,55.000000,56.000000,55.000000,62.000000,64.000000,50.000000,59.000000,...,55.000000,53.000000,57.000000,58.000000,55.000000,65.000000,58.000000,60.000000,56.000000,61.000000
3,G,9.000000,9.000000,6.000000,3.000000,6.000000,9.000000,9.000000,6.000000,12.000000,...,3.000000,9.000000,9.000000,9.000000,6.000000,3.000000,11.000000,9.000000,9.000000,9.000000
4,I,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,T,16.000000,19.000000,20.000000,17.000000,17.000000,15.000000,10.000000,18.000000,15.000000,...,22.000000,16.000000,13.000000,18.000000,20.000000,22.000000,17.000000,15.000000,16.000000,16.000000
6,S,11.000000,9.000000,7.000000,8.000000,10.000000,8.000000,9.000000,10.000000,8.000000,...,6.000000,7.000000,9.000000,13.000000,10.000000,4.000000,4.000000,11.000000,7.000000,7.000000
7,-,32.000000,37.000000,27.000000,35.000000,27.000000,30.000000,26.000000,39.000000,31.000000,...,26.000000,29.000000,33.000000,35.000000,37.000000,31.000000,28.000000,29.000000,29.000000,33.000000
8,relative_ASA,0.325164,0.337642,0.331212,0.337000,0.356303,0.352723,0.348550,0.339736,0.323166,...,0.341781,0.331213,0.364823,0.323948,0.326611,0.321532,0.339954,0.344536,0.334653,0.327027
9,phi,-82.412018,-77.747696,-77.047821,-81.664703,-79.169556,-76.990311,-76.689812,-77.660477,-81.743195,...,-78.366364,-77.611397,-78.696701,-75.616547,-78.923431,-76.008789,-80.268646,-81.748405,-78.096611,-82.545235


In [28]:
df.to_csv('/data/localhost/gordon/TNP_Project/RESULTS/greiff_results/sec_struc/VHH_TWIST_dssp.csv', index=False)